In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites.insert(0, 'All')
launch_sites

['All', 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [4]:
#the_names = pd.DataFrame({'class':[0,1],'name':['Fail','Success']}).set_index('class')
#succ_count = spacex_df[spacex_df['Launch Site'] == launch_sites[1]].groupby('class').count().join(the_names)
#succ_count = succ_count.rename(columns={'Unnamed: 0':'count'})
#px.pie(succ_count, values='count', names='name')

In [5]:
booster_df = spacex_df[['Launch Site', 'Payload Mass (kg)','Booster Version','class']].copy(deep=True)
booster_df['Booster Version Category'] = booster_df['Booster Version'].str.split(' ').str[1]
booster_df.head()

,Launch Site,Payload Mass (kg),Booster Version,class,Booster Version Category
0,CCAFS LC-40,0.0,F9 v1.0 B0003,0,v1.0
1,CCAFS LC-40,0.0,F9 v1.0 B0004,0,v1.0
2,CCAFS LC-40,525.0,F9 v1.0 B0005,0,v1.0
3,CCAFS LC-40,500.0,F9 v1.0 B0006,0,v1.0
4,CCAFS LC-40,677.0,F9 v1.0 B0007,0,v1.0


In [6]:
#px.scatter(booster_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')

In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    # dcc.Dropdown(id='site-dropdown',...)
    html.Div(
        dcc.Dropdown(launch_sites, launch_sites[0], id='site-dropdown')
    ),
    html.Br(),
    html.Div(id='dd-output-container'),
    html.Br(),
    
    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    
    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    html.Div(dcc.RangeSlider(id='payload-slider',
                             min=0, max=10000, step=1000,
                             marks={x:str(x) for x in range(0, 10001, 400)},
                             value=[min_payload, max_payload])),
    
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

In [8]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_fig(value):
    if (value == 'All') :
        pie_fig = px.pie(spacex_df[spacex_df['class'] == 1], values='class', names='Launch Site', title='Launch Success by Site')
    else :
        the_names = pd.DataFrame({'class':[0,1],'name':['Fail','Success']}).set_index('class')
        succ_count = spacex_df[spacex_df['Launch Site'] == value].groupby('class').count().join(the_names)
        succ_count = succ_count.rename(columns={'Unnamed: 0':'count'})
        pie_fig = px.pie(succ_count, values='count', names='name', color='name',
                         color_discrete_map={'Success':'#008800','Fail':'#aa0000'},
                         title='Landing Success at '+value)
    return pie_fig

In [9]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_fig(drop_value, slide_value):
    slide_min = slide_value[0]
    slide_max = slide_value[1]
    booster_df2 = booster_df[(booster_df['Payload Mass (kg)'] >= slide_min)
                             & (booster_df['Payload Mass (kg)'] <= slide_max)]
    if (drop_value == 'All') :
        scatter_fig = px.scatter(booster_df2, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                title='Launch Outcome for All Sites in the Mass Range '+str(slide_value)+' kg',
                                labels={'class':'Success'})
    else :
        booster_df3 = booster_df2[booster_df2['Launch Site'] == drop_value]
        scatter_fig = px.scatter(booster_df3, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                 title='Launch Outcome for '+str(drop_value)+' in the Mass Range '+str(slide_value)+' kg',
                                labels={'class':'Success'})
    return scatter_fig

In [10]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 12:41:50] "POST /_dash-